In [22]:
import pandas as pd

In [23]:
#https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip
input_file_name = 'covid.csv'

In [24]:
output_file_name = "covid.xlsx"
output_sheet_name = "covid"

In [25]:
df_covid = pd.read_csv(input_file_name)

/Users/fernandogonzalez/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
df_covid.head()

,numero_de_caso,fecha_apertura_snvs,fecha_toma_muestra,fecha_clasificacion,provincia,barrio,comuna,genero,edad,clasificacion,fecha_fallecimiento,fallecido,fecha_alta,tipo_contagio
0,16349158,08JUL2021:00:00:00.000000,15JUL2021:00:00:00.000000,15JUL2021:00:00:00.000000,Buenos Aires,NaN,NaN,masculino,30.0,confirmado,NaN,NaN,NaN,Comunitario
1,16371068,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,VILLA DEVOTO,11.0,femenino,39.0,confirmado,NaN,NaN,NaN,Comunitario
2,16371077,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,Buenos Aires,NaN,NaN,femenino,28.0,confirmado,NaN,NaN,NaN,Comunitario
3,16371082,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,VILLA PUEYRREDON,12.0,femenino,31.0,confirmado,NaN,NaN,NaN,Comunitario
4,16371096,08JUL2021:00:00:00.000000,19JUL2021:00:00:00.000000,19JUL2021:00:00:00.000000,Buenos Aires,NaN,NaN,masculino,32.0,confirmado,NaN,NaN,NaN,Comunitario


In [27]:
df_covid.shape

(3276519, 14)

In [28]:
df_covid.provincia.value_counts()

CABA                   1871066
Buenos Aires           1219409
Córdoba                  15897
Misiones                 15458
Chubut                   14695
Corrientes               14317
Santa Fe                 12510
Río Negro                11410
Entre Ríos                9743
Santa Cruz                9015
Mendoza                   7917
Salta                     7276
Tucumán                   7162
Neuquén                   6910
Chaco                     6876
Santiago del Estero       6378
Tierra del Fuego          5110
Formosa                   4442
Jujuy                     3775
La Pampa                  3687
San Luis                  3399
San Juan                  2981
La Rioja                  1732
Catamarca                 1583
Name: provincia, dtype: int64

In [29]:
df_covid = df_covid.loc[ df_covid.provincia == 'CABA', : ]

In [30]:
df_covid.provincia.value_counts()

CABA    1871066
Name: provincia, dtype: int64

In [31]:
df_covid['AÑO'] = [ i[5:9] for i in df_covid['fecha_clasificacion'] ]

In [32]:
df_covid = df_covid.loc[ df_covid.AÑO == '2020' ]

In [33]:
df_covid['MES'] = [ i[2:5] for i in df_covid['fecha_clasificacion'] ]

In [34]:
MESES = { 'JAN' : '1', 'FEB' : '2', 'MAR' : '3', 'APR' : '4', 'MAY' : '5', 'JUN' : '6', 'JUL' : '7', 'AUG' : '8', 'SEP' : '9', 'OCT' : '10', 'NOV' : '11', 'DEC' : '12'}

In [35]:
df_covid["MES"] = df_covid["MES"].map(MESES)

In [36]:
df_covid['TIPO'] = 'Casos de Covid'

In [37]:
df = df_covid.groupby(['AÑO','MES','TIPO'],as_index=False).count()[['AÑO','MES','TIPO','numero_de_caso']].rename({'numero_de_caso':'TOTAL'},axis=1)

In [38]:
df.MES = df.MES.astype(int)
df = df.sort_values(by=['AÑO','MES']).reset_index(drop=True)
df

,AÑO,MES,TIPO,TOTAL
0,2020,1,Casos de Covid,97
1,2020,2,Casos de Covid,30
2,2020,3,Casos de Covid,643
3,2020,4,Casos de Covid,4245
4,2020,5,Casos de Covid,13178
5,2020,6,Casos de Covid,31192
6,2020,7,Casos de Covid,49358
7,2020,8,Casos de Covid,57396
8,2020,9,Casos de Covid,52179
9,2020,10,Casos de Covid,52403


In [39]:
df['TOTAL'] = df['TOTAL'].astype(int)
df['INTERVALO']=pd.cut(df['TOTAL'],10,labels=[1,2,3,4,5,6,7,8,9,10])
df

,AÑO,MES,TIPO,TOTAL,INTERVALO
0,2020,1,Casos de Covid,97,1
1,2020,2,Casos de Covid,30,1
2,2020,3,Casos de Covid,643,1
3,2020,4,Casos de Covid,4245,1
4,2020,5,Casos de Covid,13178,2
5,2020,6,Casos de Covid,31192,4
6,2020,7,Casos de Covid,49358,7
7,2020,8,Casos de Covid,57396,8
8,2020,9,Casos de Covid,52179,7
9,2020,10,Casos de Covid,52403,7


In [40]:
enero = df.loc[ df.MES == 1, ['INTERVALO']].iloc[0,0]
diciembre = df.loc[ df.MES == 12, ['INTERVALO']].iloc[0,0]
diciembre_enero = int(((diciembre - enero)/enero)*100)
if diciembre_enero > 0:
    diciembre_enero = '+' + str(diciembre_enero) + ' %'
else:
    diciembre_enero = str(diciembre_enero) + ' %'

In [41]:
df['DICIEMBRE_ENERO'] = diciembre_enero
df

,AÑO,MES,TIPO,TOTAL,INTERVALO,DICIEMBRE_ENERO
0,2020,1,Casos de Covid,97,1,+900 %
1,2020,2,Casos de Covid,30,1,+900 %
2,2020,3,Casos de Covid,643,1,+900 %
3,2020,4,Casos de Covid,4245,1,+900 %
4,2020,5,Casos de Covid,13178,2,+900 %
5,2020,6,Casos de Covid,31192,4,+900 %
6,2020,7,Casos de Covid,49358,7,+900 %
7,2020,8,Casos de Covid,57396,8,+900 %
8,2020,9,Casos de Covid,52179,7,+900 %
9,2020,10,Casos de Covid,52403,7,+900 %


In [42]:
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

df.to_excel(writer, index=False, sheet_name=output_sheet_name)

workbook  = writer.book
worksheet = writer.sheets[output_sheet_name]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')

worksheet.set_column('A:F', 30, format)

writer.save()